In [ ]:
import tensorflow as tf
import keras
from keras.optimizers import Adam
from keras.layers import Dense
from keras.models import Model, Sequential
from keras.activations import relu, softmax
from keras.losses import SparseCategoricalCrossentropy

In [ ]:
def get_feature_extractor():
  raise NotImplementedError()

def get_internal_slicer():
  raise NotImplementedError()


def get_rx_decoder():
  raise NotImplementedError()

def phase_multiply(internally_sliced_y,h):
  raise NotImplementedError()

## PARAMS

In [ ]:
SLICED_Y_LENGTH = 16
BATCH_SIZE =  512

# in teh feature extractor path "f" : design param
# Our experiments have shown that even a
# small number of features, e.g., F = 4, significantly improves
# the performance.
N_FEATURES_EXTRACTED = 8

In [ ]:

class FeatureExtractor(Model):
    def __init__(self):
        super().__init__()

        self.cf1 = Dense(256)
        self.cf2 = Dense(N_FEATURES_EXTRACTED)

    def call(self,sliced_y):

        sliced_y = self.cf1(sliced_y)
        sliced_y = relu(sliced_y)
        sliced_y = self.cf2(sliced_y)
        return sliced_y

class PhaseEstimator(Model):
    def __init__(self):
        super().__init__()

        self.cf1 = Dense(256)
        self.cf2 = Dense(2)


    def call(self,sliced_y):
        sliced_y = self.cf1(sliced_y)
        sliced_y = relu(sliced_y)
        sliced_y = self.cf2(sliced_y)
        return sliced_y


class Rx_Decoder(Model):
    def __init__(self):
        super().__init__()

        self.cf1 = Dense(256)
        self.cf2 = Dense(256)
        self.cf3 = Dense(16)

    def call(self,concat):

        concat = self.cf1(concat)
        concat = relu(concat)
        concat = self.cf2(concat)
        concat = relu(concat)

        concat = self.cf3(concat)

        # do not use softmax here : put from logit  = True in loss func
        # concat = softmax(concat)

        return concat


class InternalSlicer(Model):
    def __init__(self,l1,l2,complex_length):
        super().__init__()

        # define the slice boundaries
        mid = complex_length // 2
        self.start = mid - l1
        self.end = mid + l2 + 1

    def call(self,sliced_y):
        return C2R(R2C(sliced_y)[:, self.start:self.end])

def R2C(a):
#     print("reached here 01")
#     print(a.shape)
    aa = tf.cast(tf.reshape(a,shape=(BATCH_SIZE,-1,2)),tf.float32)
    # print(aa)
    aaa = tf.complex(aa[:,:,0],aa[:,:,1])
    return aaa

def C2R(a):
    real, imag = tf.expand_dims(tf.math.real(a),axis=2) ,tf.expand_dims(tf.math.imag(a), axis=2)
    R = tf.concat((real,imag),axis=2)
    R = tf.reshape(R , (BATCH_SIZE,-1)  )
    return R

def phase_multiply(internally_sliced_y,estimated_phase):
    # (a,b) * (c,d) = (ac-bd,ad+bc)
    internally_sliced_y_complex = R2C(internally_sliced_y)
    estimated_phase_complex = R2C(estimated_phase)
    phase_corrected_complex = estimated_phase_complex * internally_sliced_y_complex

    phase_corrected = C2R(phase_corrected_complex)
    return phase_corrected






In [ ]:
# generate fake data
m = 512* 2** 2
X = tf.random.normal(shape=(m,SLICED_Y_LENGTH),
                     mean=0,
                     stddev=1)

Y = tf.random.uniform(shape=(m,1),
                      minval=0,
                      maxval=16,
                      dtype=tf.int32)
# Y = keras.utils.to_categorical(Y,16)


NameError: name 'tf' is not defined

In [ ]:
# sequence decoder


class SequenceDecoder(Model):

    def __init__(self,take_prev_phase_state=False):
        super().__init__()

        self.take_prev_phase_state = take_prev_phase_state

        self.feature_extractor = FeatureExtractor()
        self.phase_estimator = PhaseEstimator()
        self.internal_slicer = InternalSlicer(l1=3,l2=3,complex_length=SLICED_Y_LENGTH//2)
        self.rx_decoder = Rx_Decoder()



    def call(self,sliced_y,prev_phase_state=None):
        if self.take_prev_phase_state:
            assert prev_phase_state is not None, "RNN need the previous phase state as an input"

        extracted_features = self.feature_extractor(sliced_y)
        estimated_phase = self.phase_estimator(sliced_y)
        internally_sliced_y = self.internal_slicer(sliced_y)

#         print("estimated_phase.shape",estimated_phase.shape)
#         print("internally_sliced_y.shape",internally_sliced_y.shape)

        phase_corrected_ = phase_multiply(internally_sliced_y,estimated_phase)

        concat = tf.concat((extracted_features,phase_corrected_,prev_phase_state),axis=1)

        st_hat = self.rx_decoder(concat)

        return st_hat




In [ ]:
# test the SD

mySD =   SequenceDecoder()

mySD.compile(optimizer=Adam(learning_rate=1e-2),
             loss=SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])


mySD.fit(X,Y,epochs=2,batch_size=BATCH_SIZE)


Epoch 1/2
4/4 [==============================] - 5s 28ms/step - loss: 2.7977 - accuracy: 0.0601
Epoch 2/2
4/4 [==============================] - 0s 24ms/step - loss: 2.7524 - accuracy: 0.0815


## RNN approach for the Sequence Decoder
Reference to page 5 (Bottom right)  -> Use an RNN to pass the phase state